Loopy graphs
===

It is possible for a factor graph to contain cycles.

```
    (driver)
   -->[A]---
   |       |
   |      [+]<-[N]
   |       |
   ---[B]<--
  (inhibitor)
```

Message passing on cyclic graphs can be executed with loopy belief propagation. Loopy belief propagation is similar to sum-product message passing, but the schedule is executed recursively.

We need to set a 'breaker' message at certain interfaces to provide initial conditions for the algorithm. Since we have no knowledge about the graph beyond our breaker message, we start with a vague message that represents our ignorance. The `vague()` function can be used to produce vague (virtually uninformative) distributions.

Calculating a message within the loop
---

Suppose we want to calculate the outgoing message on the driver node, at `driver.out`. Let's first define the graph. 

In [1]:
using ForneyLab

# Define the graph
GainNode(gain=1.1, id=:driver)
GainNode(gain=0.1, id=:inhibitor)
TerminalNode(Gaussian(m=0.0, V=0.1), id=:noise)
AdditionNode(id=:add)
Edge(n(:add).i[:out], n(:inhibitor).i[:in])
Edge(n(:inhibitor).i[:out], n(:driver).i[:in])
Edge(n(:driver).i[:out], n(:add).i[:in1])
Edge(n(:noise), n(:add).i[:in2]);

The automatic sumproduct schedule generator depends on present messages in the graph. Trying to generate a sumproduct schedule in a loopy graph without predefining any breaker messages will raise an error: 

```
LoadError: Loop detected around Interface 2 (out) of ForneyLab.GainNode driver.
Consider using a loopy algorithm and setting a breaker message somewhere in this loop.
```

In [2]:
# Uncomment below to reproduce the error
# algo = SumProduct(n(:driver).i[:out])

Let's follow this advise and define a loopy sum-product algorithm.

In [5]:
algo = LoopySumProduct( n(:driver).i[:out],
                        breaker_messages=Dict(n(:driver).i[:out] => Message(vague(Gaussian))),
                        n_iterations=10)

LoopySumProduct inference algorithm
    pre-schedule length: 1
    iterative schedule length: 3
    post-schedule length: 0
    number of iterations: 10
Use show(algo.pre_schedule), show(algo.iterative_schedule), show(algo.post_schedule) to view the message passing schedules.


Now we can generate a sumproduct schedule that calculates an updated value for `driver.out`:

In [6]:
# Run the algorithm for one step
run(algo)
show(n(:driver).i[:out].message)

ForneyLab.Message{ForneyLab.Gaussian} with payload N(m=0.00, V=1.22e-03)



Calculating a message outside the loop
---

Now suppose we want to calculate the message from the addition node towards the noise node, so at interface `add.i[:in2]`. In this case, calculating the message towards `add.i[:in1]` requires the inbound message at `add.i[:out]` and vice versa, because of the loop. This indicates that we need to set breaker messages in both directions.

In [8]:
algo2 = LoopySumProduct( n(:add).i[:in2],
                         breaker_messages=Dict(
                             n(:add).i[:in1] => Message(vague(Gaussian)),
                             n(:add).i[:out] => Message(vague(Gaussian))),
                         n_iterations=10)

LoopySumProduct inference algorithm
    pre-schedule length: 1
    iterative schedule length: 6
    post-schedule length: 1
    number of iterations: 10
Use show(algo.pre_schedule), show(algo.iterative_schedule), show(algo.post_schedule) to view the message passing schedules.


In [9]:
run(algo2)
show(n(:add).i[:in2].message)

ForneyLab.Message{ForneyLab.Gaussian} with payload N(m=0.00, V=1.49e+31)

